# Прогноз продаж на январь 2025 года

<div style="max-width: 800px; margin: 0 auto; font-family: 'Segoe UI', Arial, sans-serif; line-height: 1.6; color: #333;">
    <div style="background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 4px rgba(0,0,0,0.05); margin-bottom: 30px;">
        <h2 style="color: #1976D2; margin-top: 0;">Введение</h2>
        <p style="margin-bottom: 20px;">
            Выполнен прогноз продаж на январь 2025 года, были использованы модели Prophet от Meta (Facebook). Данный анализ является базовым прогнозом, основанным на исторических данных продаж.
        </p>
        <div style=" background: #E3F2FD; border-left: 4px solid #2196F3; padding: 20px; border-radius: 4px; margin: 30px 0;">
            <p style="display: inline-block;  margin: 0; color: #1565C0; font-weight: 500;">📈 готовый pdf отчет:</p>
            <a href="https://drive.google.com/file/d/1jZIvwEfzHg445FtPhsyA0GJAjthplr7G/view?usp=sharing" style="display: inline-block;  padding-left: 12px; font-weight: 500; ">
                Открыть отчет
            </a>
        </div>
        <div style=" background: #E3F2FD; border-left: 4px solid #2196F3; padding: 20px; border-radius: 4px; margin: 30px 0;">
            <p style="display: inline-block;  margin: 0; color: #1565C0; font-weight: 500;">📈 готовый Excel отчет:</p>
            <a href="https://docs.google.com/spreadsheets/d/1LY6vLUb9e0Uis-7cS8HurltW5g8RrwGi/edit?usp=sharing&ouid=100993897678532417326&rtpof=true&sd=true" style="padding-left: 12px; display: inline-block;  font-weight: 500; ">
                Открыть отчет
            </a>
        </div>
    </div>
    <div style="background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 4px rgba(0,0,0,0.05); margin-bottom: 30px;">
        <h2 style="color: #1976D2; margin-top: 0;">Ограничения текущего анализа</h2>
        <p style="margin-bottom: 20px;">
            В рамках данного тестового задания отсутствует ряд важных факторов, которые могли бы повысить точность прогноза:
        </p>
        <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 20px;">
            <div style="background: #FAFAFA; padding: 20px; border-radius: 8px;">
                <h3 style="color: #1976D2; margin-top: 0; font-size: 1.2em;">Характеристики товаров</h3>
                <ul style="margin: 0; padding-left: 20px; color: #555;">
                    <li>Категория и подкатегория товаров</li>
                    <li>Сезонность конкретных товарных групп</li>
                    <li>Жизненный цикл товаров</li>
                </ul>
            </div>
            <div style="background: #FAFAFA; padding: 20px; border-radius: 8px;">
                <h3 style="color: #1976D2; margin-top: 0; font-size: 1.2em;">Маркетинговые данные</h3>
                <ul style="margin: 0; padding-left: 20px; color: #555;">
                    <li>История промо-акций</li>
                    <li>Маркетинговые кампании</li>
                    <li>Специальные предложения</li>
                </ul>
            </div>
            <div style="background: #FAFAFA; padding: 20px; border-radius: 8px;">
                <h3 style="color: #1976D2; margin-top: 0; font-size: 1.2em;">Внешние факторы</h3>
                <ul style="margin: 0; padding-left: 20px; color: #555;">
                    <li>Календарь праздников</li>
                    <li>Региональные особенности продаж</li>
                    <li>Активность конкурентов</li>
                </ul>
            </div>
            <div style="background: #FAFAFA; padding: 20px; border-radius: 8px;">
                <h3 style="color: #1976D2; margin-top: 0; font-size: 1.2em;">Операционные данные</h3>
                <ul style="margin: 0; padding-left: 20px; color: #555;">
                    <li>Информация о складских остатках</li>
                    <li>Данные о поставках</li>
                    <li>Логистические ограничения</li>
                </ul>
            </div>
        </div>
    </div>
    <div style="background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 4px rgba(0,0,0,0.05); margin-bottom: 30px;">
        <h2 style="color: #1976D2; margin-top: 0;">Заключение</h2>
        <p style="margin-bottom: 20px;">
            Представленный прогноз следует рассматривать как базовую модель, которая может иметь существенные отклонения от реальных показателей из-за отсутствия вышеперечисленных факторов.
        </p>
    </div>
</div>

In [2]:
import pandas as pd
import numpy as np
from prophet import Prophet
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph
from reportlab.lib.styles import ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [3]:
def prepare_data(df):
    """Подготовка данных для прогнозирования"""
    # Конвертация даты
    df['date'] = pd.to_datetime(df['date'])
    
    # Удаление дубликатов
    df = df.drop_duplicates(subset=['date', 'nm_id', 'total_price'])
    
    # Удаление отрицательных значений цен и количества
    df = df[df['total_price'] >= 0]
    
    # Группировка данных
    daily_sales = df.groupby(['date', 'nm_id']).agg({
        'total_price': ['sum', 'count']
    }).reset_index()
    
    daily_sales.columns = ['date', 'nm_id', 'total_sales', 'quantity']
    
    # Сортировка по дате
    daily_sales = daily_sales.sort_values('date')
    
    return daily_sales

In [4]:
def train_prophet_model(data, target_col):
    """Обучение модели Prophet"""
    prophet_data = data.rename(columns={'date': 'ds', target_col: 'y'})
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        seasonality_mode='multiplicative',
        interval_width=0.95
    )
    model.fit(prophet_data)
    return model

In [5]:
def generate_forecast(df):
    """Генерация прогноза на январь 2025"""
    daily_sales = prepare_data(df)
    forecasts = {}
    
    for nm_id in daily_sales['nm_id'].unique():
        try:
            item_data = daily_sales[daily_sales['nm_id'] == nm_id].copy()
            
            # Prophet прогноз для продаж и количества
            prophet_sales = train_prophet_model(item_data[['date', 'total_sales']], 'total_sales')
            prophet_quantity = train_prophet_model(item_data[['date', 'quantity']], 'quantity')
            
            # Даты для прогноза
            future_dates = pd.date_range(start='2025-01-01', end='2025-01-31', freq='D')
            prophet_future = pd.DataFrame({'ds': future_dates})
            
            # Получение прогнозов
            sales_forecast = prophet_sales.predict(prophet_future)
            quantity_forecast = prophet_quantity.predict(prophet_future)
            
            # Сохранение результатов
            forecasts[nm_id] = {
                'sales_forecast': np.sum(sales_forecast['yhat'].values),
                'quantity_forecast': np.sum(quantity_forecast['yhat'].values)
            }
            
        except Exception as e:
            print(f"Ошибка обработки артикула {nm_id}: {str(e)}")
            continue
    # Добавляем итоговую строку
    forecasts['Итог'] = {
        'sales_forecast': sum(row['sales_forecast'] for row in forecasts.values()),
        'quantity_forecast': sum(row['quantity_forecast'] for row in forecasts.values())
    }
    return forecasts

In [6]:
def create_pdf_report(forecasts):
    """Создание PDF отчета"""
    doc = SimpleDocTemplate(
        "sales_forecast_january.pdf",
        pagesize=A4,
        rightMargin=72,
        leftMargin=72,
        topMargin=72,
        bottomMargin=72
    )
    
    # Стили
    title_style = ParagraphStyle(
        'CustomTitle',
        fontName='DejaVu',
        fontSize=16,
        spaceAfter=30,
        alignment=1  # По центру
    )
    
    header_style = ParagraphStyle(
        'Header',
        fontName='DejaVu',
        fontSize=12,
        spaceAfter=20,
        alignment=1  # По центру
    )
    
    # Содержимое документа
    elements = []
    
    # Заголовок отчета
    current_date = datetime.now().strftime("%d.%m.%Y")
    elements.append(Paragraph("Анализ данных", title_style))
    elements.append(Paragraph(f"Дата: {current_date}", header_style))
    elements.append(Paragraph("Ганнова Анфиса", header_style))
    
    # Сводная таблица
    table_data = [['Артикул', 'Прогноз продаж', 'Прогноз количества']]
    for nm_id, forecast in forecasts.items():
        table_data.append([
            str(nm_id),
            f"{forecast['sales_forecast']:,.2f}",
            f"{forecast['quantity_forecast']:,.0f}"
        ])
    
    table = Table(table_data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, -1), 'DejaVu'),
        ('FONTSIZE', (0, 0), (-1, 0), 14),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),  # Белый фон для данных
        ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
        ('FONTSIZE', (0, 1), (-1, -1), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('ALIGN', (1, 1), (-1, -1), 'RIGHT'),
        ('PADDING', (0, 0), (-1, -1), 6),
    ]))
    elements.append(table)
    
    # Создание PDF
    doc.build(elements)

In [7]:
# Регистрация шрифта DejaVu
pdfmetrics.registerFont(TTFont('DejaVu', '../font/dejavu.ttf'))

In [8]:
df = pd.read_excel('wb_orders.xlsx')
print("=== ДАННЫЕ ЗАГРУЖЕНЫ ===")

=== ДАННЫЕ ЗАГРУЖЕНЫ ===


In [9]:
print(f"Загружено {len(df)} строк\n")
display(df.sample(5))

Загружено 80142 строк



,date,last_change_date,total_price,discount_percent,warehouse_name,oblast,nm_id,category,brand,is_cancel,cancel_dt,created_at,updated_at,order_type
71011,2023-07-31 19:19:00,2023-08-01,304.6,13,Электросталь,Астраханская область,95166060,Товары для животных,f486d1bc7ccbbb35,False,NaT,2023-08-01 01:59:00,2024-02-05 14:29:00,Клиентский
36984,2023-05-06 17:40:00,2023-05-06,1986.6,22,Санкт-Петербург,Новгородская область,126373749,Товары для животных,e129baf5351375dd,False,NaT,2023-05-06 22:27:00,2024-02-05 14:30:00,Клиентский
73627,2024-07-05 20:00:00,2024-07-06,384.0,20,Электросталь,Москва,95166060,Товары для животных,f486d1bc7ccbbb35,False,NaT,2024-09-06 17:06:00,2024-09-06 17:07:00,Клиентский
9957,2023-03-15 19:35:00,2023-03-15,1555.0,15,Казань,NaN,905559214,Товары для животных,e129baf5351375dd,False,NaT,2023-03-24 11:03:00,2024-02-05 14:30:00,Клиентский
37724,2022-08-24 00:00:00,2022-08-24,572.0,15,Новосёлки Чехов,Адыгея,304773881,Товары для животных,89edf2b23057232f,False,NaT,2023-02-16 13:39:00,2024-02-05 14:31:00,NaN


## Предварительная подготовка данных

In [11]:
# Просмотр информации о данных
print("=== ИНФОРМАЦИЯ О ДАННЫХ ===")
print("\nТипы данных в столбцах:")
print(df.dtypes)

=== ИНФОРМАЦИЯ О ДАННЫХ ===

Типы данных в столбцах:
date                datetime64[ns]
last_change_date    datetime64[ns]
total_price                float64
discount_percent             int64
warehouse_name              object
oblast                      object
nm_id                        int64
category                    object
brand                       object
is_cancel                     bool
cancel_dt           datetime64[ns]
created_at          datetime64[ns]
updated_at          datetime64[ns]
order_type                  object
dtype: object


In [12]:
# определяем наличие выбросов
Q1 = np.percentile(df['total_price'], 25)
Q3 = np.percentile(df['total_price'], 75)
IQR = Q3 - Q1

# Определение границ выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Поиск выбросов
outliers = [x for x in df['total_price'] if x < lower_bound or x > upper_bound]
print("Выбросы имеются: " if len(outliers) > 0 else "Выбросы отсутствуют")

Выбросы отсутствуют


In [13]:
# Проверка дубликатов
duplicates_count = df.duplicated().sum()
print(f"\nКоличество дубликатов: {duplicates_count}")


Количество дубликатов: 2797


In [14]:
# Проверка пропущенных значений
print("\nКоличество пропущенных значений по столбцам:")
missing_values = df.isnull().sum()
missing_percentages = (df.isnull().sum() / len(df)) * 100
missing_info = pd.DataFrame({
    'Пропущенные значения': missing_values,
    'Процент пропущенных': missing_percentages.round(2)
})
print(missing_info)


Количество пропущенных значений по столбцам:
                  Пропущенные значения  Процент пропущенных
date                                 0                 0.00
last_change_date                     0                 0.00
total_price                          0                 0.00
discount_percent                     0                 0.00
warehouse_name                       0                 0.00
oblast                            2964                 3.70
nm_id                                0                 0.00
category                             0                 0.00
brand                                0                 0.00
is_cancel                            0                 0.00
cancel_dt                        76240                95.13
created_at                         536                 0.67
updated_at                           0                 0.00
order_type                       14570                18.18


#### Наблюдаем в некоторых столбцах присутствуют пропущенные значения. Однако количество пропущеных значений в этих столбцах не является значительным и не влияет на общую целостность данных. Следовательно, обработка пропущенных значений не представляет необходимой. Поработаем с дубликатами.

In [16]:
# Удаление дубликатов
df_cleaned = df.drop_duplicates()
print(f"\nУдалено дубликатов: {len(df) - len(df_cleaned)}")

print("\n=== ИТОГОВАЯ ИНФОРМАЦИЯ ===")
print(f"Исходное количество строк: {len(df)}")
print(f"Количество строк после очистки: {len(df_cleaned)}")
# будем использовать далее очищенный файл
df = df_cleaned


Удалено дубликатов: 2797

=== ИТОГОВАЯ ИНФОРМАЦИЯ ===
Исходное количество строк: 80142
Количество строк после очистки: 77345


## Далее рассчитаем прогноз на январь месяц

In [18]:
forecasts = generate_forecast(df)

14:10:34 - cmdstanpy - INFO - Chain [1] start processing
14:10:37 - cmdstanpy - INFO - Chain [1] done processing
14:10:37 - cmdstanpy - INFO - Chain [1] start processing
14:10:38 - cmdstanpy - INFO - Chain [1] done processing
14:10:39 - cmdstanpy - INFO - Chain [1] start processing
14:10:41 - cmdstanpy - INFO - Chain [1] done processing
14:10:42 - cmdstanpy - INFO - Chain [1] start processing
14:10:44 - cmdstanpy - INFO - Chain [1] done processing
14:10:45 - cmdstanpy - INFO - Chain [1] start processing
14:10:46 - cmdstanpy - INFO - Chain [1] done processing
14:10:47 - cmdstanpy - INFO - Chain [1] start processing
14:10:49 - cmdstanpy - INFO - Chain [1] done processing
14:10:50 - cmdstanpy - INFO - Chain [1] start processing
14:10:51 - cmdstanpy - INFO - Chain [1] done processing
14:10:52 - cmdstanpy - INFO - Chain [1] start processing
14:10:54 - cmdstanpy - INFO - Chain [1] done processing
14:10:54 - cmdstanpy - INFO - Chain [1] start processing
14:10:55 - cmdstanpy - INFO - Chain [1]

In [19]:
# Создание PDF отчета
create_pdf_report(forecasts)

In [20]:
print("Отчет успешно создан: sales_forecast_january.pdf")

Отчет успешно создан: sales_forecast_january.pdf


In [22]:
data = []
for nm_id, forecast in forecasts.items():
    data.append({
        'Артикул': nm_id,
        'Прогноз продаж': f"{forecast['sales_forecast']:,.2f}",
        'Прогноз количества': f"{forecast['quantity_forecast']:,.0f}"
    })

df_report = pd.DataFrame(data)
display(df_report)

,Артикул,Прогноз продаж,Прогноз количества
0,-83054245,"127,106.80",31
1,95166060,"13,939.50",31
2,126373749,"100,440.90",30
3,304773881,"42,002.47",30
4,440376223,"32,038.66",31
5,678167538,"152,024.62",30
6,879714109,"17,418.74",31
7,905559214,"61,706.79",31
8,811003631,"79,555.72",31
9,44403861,"17,428.88",31


In [43]:
# экспорт Excel файла
df_report.to_excel("sales_forecast_january.xlsx")